# Neo4j Graph Database for Building Structure

This notebook demonstrates how to connect to a Neo4j graph database and create a dynamic graph representing a building structure based on data from `id_mapping.csv`.

The CSV contains:
- room: The room number
- floor: The floor number  
- id: Base ID for generating sensor IDs

For each room, we create 4 sensors (Heat, Light, CO2, Motion) with IDs generated as:
- Format: [First letter of sensor type][Base ID * 4 + sensor index + 1]
- Example: For base ID 1, Heat sensor = H5, Light sensor = L6, CO2 sensor = C7, Motion sensor = M8

We'll implement:
1. A function to execute Neo4j queries
2. Methods to create nodes (floors, rooms, sensors) based on CSV data
3. Methods to create relationships (edges) between nodes
4. Dynamic sensor ID generation based on the CSV mapping

## Setup and Connection

First, let's import necessary libraries and establish a connection to the Neo4j database.

In [83]:
# Import required libraries
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
import pandas as pd

# Load environment variables
load_dotenv()

# Neo4j connection parameters - update these or add to .env file
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")

# Class to handle Neo4j database operations
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.uri = uri
        self.user = user
        self.password = password
        self.driver = None
        try:
            self.driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))
            print("Connected to Neo4j database!")
        except Exception as e:
            print(f"Failed to connect to Neo4j database: {e}")
    
    def close(self):
        if self.driver is not None:
            self.driver.close()
            print("Connection to Neo4j closed.")
    
    def execute_query(self, query, parameters=None):
        """
        Execute a Cypher query against the Neo4j database
        
        Args:
            query (str): The Cypher query to execute
            parameters (dict, optional): Parameters for the query
            
        Returns:
            list: Results of the query
        """
        assert self.driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try:
            session = self.driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print(f"Query failed: {e}")
        finally:
            if session is not None:
                session.close()
        
        return response

In [84]:
id_mapping = pd.read_csv('id_mapping.csv')
print("ID Mapping Data:")
print(id_mapping.head())
print(f"\nTotal records: {len(id_mapping)}")
print(f"Unique floors: {sorted(id_mapping['floor'].unique())}")
print(f"Unique rooms: {sorted(id_mapping['room'].unique())}")

ID Mapping Data:
   room  floor  id
0   413      4   1
1   415      4   2
2   417      4   3
3   510      5   4
4   511      5   5

Total records: 9
Unique floors: [np.int64(4), np.int64(5), np.int64(6)]
Unique rooms: [np.int64(413), np.int64(415), np.int64(417), np.int64(510), np.int64(511), np.int64(513), np.int64(621), np.int64(640), np.int64(644)]


In [85]:
# Demonstrate the sensor ID generation logic
def preview_sensor_ids(id_mapping_df):
    """
    Preview how sensor IDs will be generated based on the CSV data
    """
    sensor_types = ["Heat", "Light", "CO2", "Motion"]
    preview_data = []
    
    print("Sensor ID Generation Preview:")
    print("Format: [SensorType_FirstLetter][BaseID * 4 + SensorIndex + 1]")
    print("\nExample sensor IDs for each room:")
    
    for _, row in id_mapping_df.head(3).iterrows():  # Show first 3 rooms as example
        room_number = int(row['room'])
        base_id = int(row['id'])
        floor_number = int(row['floor'])
        
        print(f"\nRoom {room_number} (Floor {floor_number}, Base ID: {base_id}):")
        
        for i, sensor_type in enumerate(sensor_types):
            sensor_id = f"{sensor_type[0]}{base_id * 4 + i + 1}"
            print(f"  {sensor_type}: {sensor_id}")
            preview_data.append({
                'Room': room_number,
                'Floor': floor_number,
                'SensorType': sensor_type,
                'SensorID': sensor_id
            })
    
    return pd.DataFrame(preview_data)

# Preview the sensor ID generation
sensor_preview = preview_sensor_ids(id_mapping)
sensor_preview

Sensor ID Generation Preview:
Format: [SensorType_FirstLetter][BaseID * 4 + SensorIndex + 1]

Example sensor IDs for each room:

Room 413 (Floor 4, Base ID: 1):
  Heat: H5
  Light: L6
  CO2: C7
  Motion: M8

Room 415 (Floor 4, Base ID: 2):
  Heat: H9
  Light: L10
  CO2: C11
  Motion: M12

Room 417 (Floor 4, Base ID: 3):
  Heat: H13
  Light: L14
  CO2: C15
  Motion: M16


,Room,Floor,SensorType,SensorID
0,413,4,Heat,H5
1,413,4,Light,L6
2,413,4,CO2,C7
3,413,4,Motion,M8
4,415,4,Heat,H9
5,415,4,Light,L10
6,415,4,CO2,C11
7,415,4,Motion,M12
8,417,4,Heat,H13
9,417,4,Light,L14


In [86]:
# Display summary statistics of the CSV data
print("=== Building Data Summary ===")
print(f"Total rooms: {len(id_mapping)}")
print(f"Floors present: {sorted(id_mapping['floor'].unique())}")
print(f"Room numbers: {sorted(id_mapping['room'].unique())}")
print(f"ID range: {id_mapping['id'].min()} to {id_mapping['id'].max()}")
print(f"\nRooms per floor:")
floor_counts = id_mapping.groupby('floor').size().sort_index()
for floor, count in floor_counts.items():
    print(f"  Floor {floor}: {count} rooms")
    
print(f"\nTotal sensors that will be created: {len(id_mapping) * 4}")
print(f"Sensor types per room: Heat, Light, CO2, Motion")
print(f"\nFirst few rows of data:")
id_mapping.head(10)

=== Building Data Summary ===
Total rooms: 9
Floors present: [np.int64(4), np.int64(5), np.int64(6)]
Room numbers: [np.int64(413), np.int64(415), np.int64(417), np.int64(510), np.int64(511), np.int64(513), np.int64(621), np.int64(640), np.int64(644)]
ID range: 1 to 9

Rooms per floor:
  Floor 4: 3 rooms
  Floor 5: 3 rooms
  Floor 6: 3 rooms

Total sensors that will be created: 36
Sensor types per room: Heat, Light, CO2, Motion

First few rows of data:


,room,floor,id
0,413,4,1
1,415,4,2
2,417,4,3
3,510,5,4
4,511,5,5
5,513,5,6
6,621,6,7
7,640,6,8
8,644,6,9


## Connect to the Database

Let's instantiate a connection to our Neo4j database.

In [87]:
# Create a connection to the Neo4j database
conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

Connected to Neo4j database!


## Create Graph Schema

Now let's create constraints to ensure uniqueness of our nodes.

In [88]:
# Create constraints for unique nodes
constraints = [
    "CREATE CONSTRAINT floor_number IF NOT EXISTS FOR (f:Floor) REQUIRE f.floor_number IS UNIQUE",
    "CREATE CONSTRAINT room_number IF NOT EXISTS FOR (r:Room) REQUIRE r.room_number IS UNIQUE",
    "CREATE CONSTRAINT sensor_id IF NOT EXISTS FOR (s:Sensor) REQUIRE s.sensor_id IS UNIQUE"
]

for constraint in constraints:
    conn.execute_query(constraint)
    
print("Constraints created successfully!")

Constraints created successfully!


## Create Floor Nodes

First, let's create the floor nodes.

In [89]:
# Function to create floor nodes based on CSV data
def create_floor_nodes(conn, id_mapping_df):
    query = """
    UNWIND $floors AS floor
    MERGE (f:Floor {floor_number: floor.number})
    RETURN f
    """
    
    # Get unique floors from the CSV
    unique_floors = id_mapping_df['floor'].unique()
    floors = [{"number": int(floor)} for floor in unique_floors]
    
    result = conn.execute_query(query, {"floors": floors})
    
    print(f"Created {len(floors)} floor nodes: {sorted([f['number'] for f in floors])}")
    return result

# Create floor nodes
floor_nodes = create_floor_nodes(conn, id_mapping)

Created 3 floor nodes: [4, 5, 6]


## Create Room Nodes and Connect to Floors

Next, let's create room nodes for each floor and connect them to their respective floors.

In [90]:
# Function to create room nodes and connect them to floors based on CSV data
def create_room_nodes(conn, id_mapping_df):
    query = """
    UNWIND $rooms AS room
    MATCH (f:Floor {floor_number: room.floor})
    MERGE (r:Room {room_number: room.number})
    MERGE (f)-[:HAS_ROOM]->(r)
    RETURN f, r
    """
    
    # Get room and floor data from CSV
    rooms = []
    for _, row in id_mapping_df.iterrows():
        rooms.append({
            "floor": int(row['floor']),
            "number": int(row['room'])
        })
    
    result = conn.execute_query(query, {"rooms": rooms})
    
    print(f"Created {len(rooms)} room nodes and connected them to floors")
    print(f"Rooms created: {sorted([r['number'] for r in rooms])}")
    return result

# Create room nodes
room_nodes = create_room_nodes(conn, id_mapping)

Created 9 room nodes and connected them to floors
Rooms created: [413, 415, 417, 510, 511, 513, 621, 640, 644]


## Create Sensor Nodes and Connect to Rooms

Finally, let's create sensor nodes of various types and connect them to their respective rooms.

In [91]:
# Function to create sensor nodes and connect them to rooms based on CSV data
def create_sensor_nodes(conn, id_mapping_df):
    query = """
    UNWIND $sensors AS sensor
    MATCH (r:Room {room_number: sensor.room})
    MERGE (s:Sensor {
        sensor_id: sensor.id,
        type: sensor.type
    })
    MERGE (r)-[:HAS_SENSOR]->(s)
    RETURN r, s
    """
    
    sensor_types = ["tempreture", "light", "co2", "motion"]
    sensors = []
    
    # Create sensors for each room using the CSV data
    for _, row in id_mapping_df.iterrows():
        room_number = int(row['room'])
        base_id = int(row['id'])
        
        for i, sensor_type in enumerate(sensor_types):
            # Generate sensor ID: first letter of sensor type + (base_id * 4 + sensor_index)
            sensor_id = f"{sensor_type[0]}{base_id}"
            sensors.append({
                "id": sensor_id,
                "type": sensor_type,
                "room": room_number
            })
    
    result = conn.execute_query(query, {"sensors": sensors})
    
    print(f"Created {len(sensors)} sensor nodes and connected them to rooms")
    print(f"Sensor types per room: {sensor_types}")
    print(f"Sample sensor IDs: {[s['id'] for s in sensors[:8]]}...")  # Show first 8 sensor IDs
    return result

# Create sensor nodes
sensor_nodes = create_sensor_nodes(conn, id_mapping)

Created 36 sensor nodes and connected them to rooms
Sensor types per room: ['tempreture', 'light', 'co2', 'motion']
Sample sensor IDs: ['t1', 'l1', 'c1', 'm1', 't2', 'l2', 'c2', 'm2']...


## Query the Graph

Let's verify our graph structure by running some queries.

In [92]:
# Count nodes by type
def count_nodes(conn):
    query = """
    MATCH (n)
    RETURN labels(n) AS label, count(*) AS count
    """
    
    result = conn.execute_query(query)
    
    # Convert to DataFrame for better visualization
    data = [{"Label": r["label"][0], "Count": r["count"]} for r in result]
    df = pd.DataFrame(data)
    
    return df

# Count nodes by type
node_counts = count_nodes(conn)
node_counts

,Label,Count
0,Floor,3
1,Room,9
2,Sensor,36


In [93]:
# Query structure of a specific floor
def query_floor_structure(conn, floor_number):
    query = """
    MATCH (f:Floor {floor_number: $floor_number})-[:HAS_ROOM]->(r:Room)-[:HAS_SENSOR]->(s:Sensor)
    RETURN f.floor_number AS Floor, r.room_number AS Room, s.sensor_id AS SensorID, s.type AS SensorType
    ORDER BY r.room_number, s.type
    """
    
    result = conn.execute_query(query, {"floor_number": floor_number})
    
    # Convert to DataFrame
    data = [{
        "Floor": r["Floor"],
        "Room": r["Room"],
        "SensorID": r["SensorID"],
        "SensorType": r["SensorType"]
    } for r in result]
    
    df = pd.DataFrame(data)
    
    return df

# Get the first floor from our data to query
first_floor = sorted(id_mapping['floor'].unique())[0]
print(f"Querying structure for floor {first_floor}:")
floor_structure = query_floor_structure(conn, first_floor)
floor_structure

Querying structure for floor 4:


,Floor,Room,SensorID,SensorType
0,4,413,c1,co2
1,4,413,l1,light
2,4,413,m1,motion
3,4,413,t1,tempreture
4,4,415,c2,co2
5,4,415,l2,light
6,4,415,m2,motion
7,4,415,t2,tempreture
8,4,417,c3,co2
9,4,417,l3,light


## Visualize the Graph (Optional)

Neo4j Browser provides a great visualization tool. Here's a Cypher query you can run in the Neo4j Browser to visualize the entire graph:

```
MATCH p=(f:Floor)-[:HAS_ROOM]->(r:Room)-[:HAS_SENSOR]->(s:Sensor)
RETURN p LIMIT 25
```

## Advanced Queries

Let's run a few more advanced queries to demonstrate the power of graph databases.

In [94]:
# Find all heat sensors in the building
def query_heat_sensors(conn):
    query = """
    MATCH (f:Floor)-[:HAS_ROOM]->(r:Room)-[:HAS_SENSOR]->(s:Sensor {type: 'Heat'})
    RETURN f.floor_number AS Floor, r.room_number AS Room, s.sensor_id AS SensorID
    ORDER BY f.floor_number, r.room_number
    """
    
    result = conn.execute_query(query)
    
    # Convert to DataFrame
    data = [{
        "Floor": r["Floor"],
        "Room": r["Room"],
        "SensorID": r["SensorID"]
    } for r in result]
    
    df = pd.DataFrame(data)
    
    return df

# Query heat sensors
heat_sensors = query_heat_sensors(conn)
heat_sensors

""


In [95]:
# Find how many sensors of each type exist per floor
def query_sensor_counts_by_floor(conn):
    query = """
    MATCH (f:Floor)-[:HAS_ROOM]->(r:Room)-[:HAS_SENSOR]->(s:Sensor)
    RETURN f.floor_number AS Floor, s.type AS SensorType, count(s) AS Count
    ORDER BY f.floor_number, s.type
    """
    
    result = conn.execute_query(query)
    
    # Convert to DataFrame
    data = [{
        "Floor": r["Floor"],
        "SensorType": r["SensorType"],
        "Count": r["Count"]
    } for r in result]
    
    df = pd.DataFrame(data)
    
    return df

# Query sensor counts by floor
sensor_counts = query_sensor_counts_by_floor(conn)
sensor_counts

,Floor,SensorType,Count
0,4,co2,3
1,4,light,3
2,4,motion,3
3,4,tempreture,3
4,5,co2,3
5,5,light,3
6,5,motion,3
7,5,tempreture,3
8,6,co2,3
9,6,light,3


## Clean Up (Optional)

If you want to delete the entire graph and start over, you can run the following command:

In [ ]:
# Delete all nodes and relationships in the database
def delete_all(conn):
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    
    conn.execute_query(query)
    print("All nodes and relationships have been deleted.")

# Uncomment the next line to delete all data
# delete_all(conn)

All nodes and relationships have been deleted.


## Close Connection

Remember to close the Neo4j connection when you're done.

In [54]:
# Close the connection
conn.close()

Connection to Neo4j closed.
